<a href="https://colab.research.google.com/github/imrohu/NLP/blob/master/NLPAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras 
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

url='https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
data=pd.read_csv(url, sep='\t', header=0, quoting=3)

data.head(100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
95,96,3,", I suspect ,",2
96,97,3,"I suspect ,",2
97,98,3,I suspect,2
98,99,3,I,2


In [0]:
data.shape

(156060, 4)

In [0]:
import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
pd.set_option('max_colwidth',400)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data['cleaned_reviews'] = cleaned_reviews

data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [0]:
from keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

X = data.cleaned_reviews.values
Y = to_categorical(data.Sentiment.values)

vectorizer = TfidfVectorizer(stop_words = None, ngram_range=(1,2))

from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3, random_state=2003)


In [0]:
from nltk import FreqDist
all_words=' '.join(X_train)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)

num_unique_word=len(dist)
num_unique_word

13725

In [0]:

import torch
torch.manual_seed(2020)
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu, softmax, sigmoid
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import SGD, Adam
from torch.nn import L1Loss, CrossEntropyLoss

In [0]:

class CnnClassifier(torch.nn.Module):
  def __init__(self, batch_size, inputs, outputs):

    super(CnnClassifier, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs
    self.input_layer = Conv1d(inputs, batch_size, 1)
    self.max_pooling_layer = MaxPool1d(1)
    self.conv_layer = Conv1d(batch_size, 128, 1)
    self.flatten_layer = Flatten()
    self.linear_layer = Linear(128, 64)
    self.output_layer = Linear(64, outputs)
  def feed(self, input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))

    output = self.max_pooling_layer(output)

    output = relu(self.conv_layer(output))

    output = self.flatten_layer(output)

    output = self.linear_layer(output)

    output = self.output_layer(output)
  
    output = softmax(output)
   
    return output

In [0]:
def categorical_accuracy(preds, y):
  
    max_preds = preds.argmax(dim = 1, keepdim = True)
    correct = max_preds.squeeze(1).eq(y)
    return (correct.sum() / torch.FloatTensor([y.shape[0]])) / torch.FloatTensor([y.shape[0]])

In [0]:

def model_loss(model, dataset, train = False, optimizer = None):

  performance = L1Loss()
  criterion = CrossEntropyLoss()
  avg_loss = 0
  avg_accu = 0
  count = 0
  
  for input, output in iter(dataset):
    predictions = model.feed(input)
    
    loss = performance(predictions, output)
    
    tmp_accu = categorical_accuracy(predictions, output)
    
    if(train):
      optimizer.zero_grad()

      loss.backward()
      optimizer.step()
    avg_loss += loss.item()

    avg_accu += tmp_accu.item()
    count += 1
    
  return avg_loss / count, avg_accu / count

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
batch_size = 128

x_train = vectorizer.fit_transform(X_train)
model = CnnClassifier(batch_size, x_train.shape[1], 5)

model.cuda()

CnnClassifier(
  (input_layer): Conv1d(88931, 128, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=5, bias=True)
)

In [0]:
r_len=[]
for text in X_train:
    word=word_tokenize(text)
    l=len(word)
    
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN


48

In [0]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 256
epochs=5

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_features)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

X_test = tokenizer.texts_to_sequences(data)

In [0]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
print(X_train.shape,X_val.shape)

(109242, 48) (46818, 48)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [0]:
model2 = Sequential()

model2.add(Embedding(max_features, 150, input_length=max_words))

model2.add(SpatialDropout1D(0.2))

model2.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Flatten())

model2.add(Dense(5, activation='softmax'))

In [0]:
import torch

torch.save(model, '/content/gdrive/My Drive/Assignment2.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CnnClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
model = torch.load('/content/gdrive/My Drive/Assignment2.pt')
#model.eval()

In [0]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 109242 samples, validate on 46818 samples
Epoch 1/5
109242/109242 [==============================] - 3s 30us/step - loss: 1.2922 - acc: 0.5089 - val_loss: 1.2825 - val_acc: 0.5123
Epoch 2/5
109242/109242 [==============================] - 3s 26us/step - loss: 1.2865 - acc: 0.5089 - val_loss: 1.2817 - val_acc: 0.5123
Epoch 3/5
109242/109242 [==============================] - 3s 27us/step - loss: 1.2862 - acc: 0.5089 - val_loss: 1.2824 - val_acc: 0.5123
Epoch 4/5
109242/109242 [==============================] - 3s 27us/step - loss: 1.2860 - acc: 0.5089 - val_loss: 1.2829 - val_acc: 0.5123
Epoch 5/5
109242/109242 [==============================] - 3s 26us/step - loss: 1.2858 - acc: 0.5089 - val_loss: 1.2824 - val_acc: 0.5123
